# Basic Diffusivity Analysis Tutorial

This notebook demonstrates how to use the garnet-electrolyte-analysis package to:
1. Load molecular dynamics trajectory data
2. Calculate Li-ion diffusivity
3. Analyze other element stability
4. Create publication-quality plots

## Prerequisites

- LAMMPS dump file (trajectory)
- LAMMPS data file (initial structure)
- Python environment with the package installed

In [ ]:
import garnet_analysis as ga
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

## 1. Load and Analyze Trajectory Data

First, we'll load a LAMMPS trajectory and calculate Li-ion diffusivity.

In [ ]:
# Example file paths - replace with your actual data
dump_file = "path/to/your/dump.traj"
data_file = "path/to/your/structure.data"
temperature = 1000  # K

# Create analyzer from LAMMPS files
analyzer = ga.DiffusivityAnalyzer.from_lammps_dump(
    dump_path=dump_file,
    data_path=data_file,
    time_step=1.0,  # fs
    step_skip=500
)

print(f"Loaded trajectory with {len(analyzer.structures)} frames")

## 2. Calculate Li-ion Diffusivity

In [ ]:
# Calculate Li-ion diffusivity
li_result = analyzer.calculate_diffusivity(
    species='Li',
    temperature=temperature
)

print(f"Li-ion diffusivity: {li_result.diffusivity:.3e} ± {li_result.std_error:.3e} cm²/s")
print(f"Analysis time range: {li_result.time_range[0]:.1f} - {li_result.time_range[1]:.1f} ps")
print(f"R² of linear fit: {li_result.r_squared:.3f}")

## 3. Analyze Other Elements for Melting Detection

In [ ]:
# Analyze all non-Li elements
other_elements = analyzer.analyze_all_elements(temperature=temperature)

print("Element stability analysis:")
for element, data in other_elements.items():
    print(f"{element}: MSD_final = {data.msd_final:.3f} Å² (threshold: 20 Å²)")
    status = "STABLE" if data.msd_final < 20 else "MELTED"
    print(f"  Status: {status}")

## 4. Create MSD Plot

Generate a publication-quality mean square displacement plot.

In [ ]:
# For demonstration, create synthetic MSD data
# In practice, this would come from your AIMD analysis
time_ps = np.linspace(0, 100, len(analyzer.structures))
msd_synthetic = li_result.msd_slope * time_ps + np.random.normal(0, 0.5, len(time_ps))

# Create MSD plot
fig = ga.plot_msd(
    time_ps=time_ps,
    msd=msd_synthetic,
    diffusivity_result=li_result,
    other_elements=other_elements,
    title="Li-ion MSD Analysis",
    save_path="msd_plot.png"
)

plt.show()

## 5. Compare with Reference Materials

Compare your results with known LLZO reference values.

In [ ]:
# Get reference diffusivity values
tllzo_1000k = ga.get_reference_diffusivity('tetragonal', 1000)
cllzo_1000k = ga.get_reference_diffusivity('cubic', 1000)

print("Reference values at 1000K:")
print(f"Tetragonal LLZO: {tllzo_1000k[1000]['value']:.3e} cm²/s")
print(f"Cubic LLZO: {cllzo_1000k[1000]['value']:.3e} cm²/s")
print(f"Your material: {li_result.diffusivity:.3e} cm²/s")

# Calculate relative performance
tllzo_ratio = li_result.diffusivity / tllzo_1000k[1000]['value']
print(f"\nYour material is {tllzo_ratio:.1f}x the diffusivity of tLLZO")

## 6. Screening Criteria Assessment

Apply standard screening criteria to determine if the material passes.

In [ ]:
# Define screening criteria
criteria = {
    'min_diffusivity': 0.9 * tllzo_1000k[1000]['value'],  # 90% of tLLZO
    'max_msd': 20.0,  # Å²
    'min_r_squared': 0.8
}

# Check criteria
passes_diffusivity = li_result.diffusivity >= criteria['min_diffusivity']
passes_stability = all(data.msd_final < criteria['max_msd'] for data in other_elements.values())
passes_fit_quality = li_result.r_squared >= criteria['min_r_squared']

print("Screening Results:")
print(f"✓ Diffusivity >= 90% tLLZO: {passes_diffusivity}")
print(f"✓ All elements stable (MSD < 20 Å²): {passes_stability}")
print(f"✓ Good fit quality (R² >= 0.8): {passes_fit_quality}")

overall_pass = passes_diffusivity and passes_stability and passes_fit_quality
print(f"\nOverall result: {'PASS' if overall_pass else 'FAIL'}")

## 7. Save Results

Export results for further analysis or reporting.

In [ ]:
# Create results dictionary
results = {
    'material_name': 'Your_Material_Name',
    'temperature': temperature,
    'diffusivity': li_result.diffusivity,
    'diffusivity_std': li_result.std_error,
    'r_squared': li_result.r_squared,
    'time_range_start': li_result.time_range[0],
    'time_range_end': li_result.time_range[1],
    'screening_pass': overall_pass
}

# Add element stability data
for element, data in other_elements.items():
    results[f'{element}_msd_final'] = data.msd_final
    results[f'{element}_stable'] = data.msd_final < 20.0

# Convert to DataFrame and save
df_results = pd.DataFrame([results])
df_results.to_csv('analysis_results.csv', index=False)

print("Results saved to 'analysis_results.csv'")
print("\nSummary:")
print(df_results.to_string(index=False))

## Next Steps

After completing this basic analysis, you can:

1. **Temperature Series Analysis**: Use `02_arrhenius_analysis.ipynb` to analyze multiple temperatures
2. **High-throughput Screening**: Use `03_screening_workflow.ipynb` for batch processing
3. **Advanced Plotting**: Explore more visualization options in the plotting modules
4. **Energy Analysis**: Combine with formation energy calculations for comprehensive screening

## Troubleshooting

Common issues and solutions:

- **PyMatGen ImportError**: Install with `conda install pymatgen`
- **AIMD ImportError**: The package falls back to basic methods without AIMD
- **File not found**: Check file paths and ensure files are accessible
- **Poor fit quality**: Adjust analysis parameters or check trajectory quality